# 1. Import Libraries

In [43]:
#import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from datetime import datetime
from sklearn.model_selection import train_test_split
from tensorflow.keras import Input, Model
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet201
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50
from tensorflow.keras.layers import Add, AveragePooling2D, BatchNormalization, Conv2D, Dense, Dropout, Flatten
from tensorflow.keras.layers import GlobalAveragePooling2D, Layer, MaxPool2D, ReLU, Resizing
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.regularizers import L2
from tqdm import tqdm
from sklearn.metrics import accuracy_score

# 2. Data Loading and Preprocessing 

## 2.1 Segment protrait
Selfie segmentation model can segment the portrait of a person, and can be used for replacing or modifying the background in an image.

In [ ]:
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_selfie_segmentation = mp.solutions.selfie_segmentation

file_list = glob.glob(image_path + '/*.jpg')

BG_COLOR = (255, 255, 255)
with mp_selfie_segmentation.SelfieSegmentation(model_selection = 0) as selfie_segmentation:
    for i in tqdm(range(len(file_list))):
        file_name = file_list[i].split('/')[-1]
        image = cv2.imread(file_list[i])
        image_height, image_width, _ = image.shape

        results = selfie_segmentation.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
        
        bg_image = np.zeros(image.shape, dtype=np.uint8)
        bg_image[:] = BG_COLOR

        output_image = np.where(condition, image, bg_image)
        
        # cleaned images are under folder /DSA5204_FAP_Project/Datasets/SCUT-FBP/Cleaned_Images
        cv2.imwrite(dataset_path + '/Cleaned_Images/' + file_name, output_image)

## 2.2 Load cleaned image

In [3]:
#proj_path = '/content/drive/MyDrive/DSA5204_FAP_Project'
#dataset_path = proj_path + '/Datasets/SCUT-FBP'
image_path = '/users/xincai.zhang/Downloads/Cleaned_Images/'
rate_path = '/users/xincai.zhang/Downloads/All_labels.txt'
model_path =  './MT-Resnet/'

In [4]:
# load data: y_arr = [rating, race, gender]
def load_data(img_dir, label_dir):
    all_ratings = pd.read_csv(label_dir, sep = ' ', header = None)
    all_ratings.columns = ['img_path', 'rating']

    img_arr = np.zeros([len(all_ratings), 64, 64, 3])
    y_arr = np.zeros([len(all_ratings), 3])

    for i in tqdm(range(len(all_ratings))):
    #for i in tqdm(range(500)):
        file_name = all_ratings.iloc[i, 0]
        race = file_name[0]
        gender = file_name[1]
        if race == 'A':
            y_arr[i, 1] = 0
        else:
            y_arr[i, 1] = 1

        if gender == 'M':
            y_arr[i, 2] = 0
        else:
            y_arr[i, 2] = 1 
        
        y_arr[i, 0] = all_ratings.iloc[i, 1]
    
        img = tf.io.read_file(img_dir + file_name)
        img = tf.image.decode_jpeg(img, channels = 3)
        img = tf.keras.layers.Resizing(64, 64)(img)
        img = preprocess_input(img)
        img_arr[i] = img

    return img_arr, y_arr

img_arr, y_arr = load_data(image_path, rate_path)

  0%|                                                                                                                                                                                                                                | 0/5500 [00:00<?, ?it/s]2023-04-21 11:57:24.203123: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5500/5500 [00:22<00:00, 245.26it/s]


## 2.3 Split Dataset

In [19]:
img_full_train, img_test, y_full_train, y_test = train_test_split(img_arr, y_arr, stratify = y_arr[:, 2], test_size = 0.4, 
                                                                  random_state = 0)

# 3. Build Model

## 3.1 MT-Restnet model
To adapt to FAP learning and multi-task requirement, the proposed model of MT-Resnet consists the following changes:
- Input streaming layer is modified to three 3x3 convolution layers, instead of a 7x7 convolution layer, to reduce the training cost significantly. 
- Backbone of resnet50 consists of 4 stages, and each stage includes a  down-sampling block and several residual blocks based on the sequence of stages. 
- For the FAP task, the output part is implemented using a global average pooling layer followed by a fully connected layer. Linear activation function is utilized to predict facial attractiveness score.
- For a gender prediction task, the structure of the output part is similar, whereas the activation function is sigmoid for the classification task.



In [6]:
class Input_Stream(Model):
    def __init__(self, num_maps, kernel_size, momentum_parameter, pool):
        super(Input_Stream, self).__init__()
        
        self.conv1 = Conv2D(num_maps, kernel_size, strides = 2, use_bias = False)
        self.bn1 = BatchNormalization(momentum = momentum_parameter)
        self.conv2 = Conv2D(num_maps, kernel_size, padding = 'same', use_bias = False)
        self.bn2 = BatchNormalization(momentum = momentum_parameter)
        self.conv3 = Conv2D(num_maps, kernel_size, padding = 'same', use_bias = False)
        self.bn3 = BatchNormalization(momentum = momentum_parameter)
        self.relu = ReLU()
        self.max_pool = MaxPool2D(pool_size = pool, strides = 2)
        
       
    
    def call(self, inputs):
        x = self.relu(self.bn1(self.conv1(inputs)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.max_pool(x)
        
        return x
    
    
class MT_ResNet(Model):
    def __init__(self):
        super(MT_ResNet, self).__init__()
        
        self.start_block = Input_Stream(num_maps = 64, kernel_size = 3, momentum_parameter = 0.5, pool = 3)
        self.resnet50 = ResNet50(include_top = False, pooling = 'avg')
        self.resnet50_backbone = Model(self.resnet50.layers[7].input, self.resnet50.layers[-1].output)
        self.fc1 = Dense(1, name = 'fap', use_bias = False)
        self.fc2 = Dense(1, activation = 'sigmoid', name = 'gender', use_bias = False)
        
        
        
    def call(self, inputs):
        x = self.start_block(inputs)
        x = self.resnet50_backbone(x)
        x_fap = self.fc1(x)
        x_gender = self.fc2(x)
        
        return x_fap, x_gender

    

## 3.2 Callbacks used to prevent overfitting
- Early stopping
- Learning rate scheduler
- Reduce learning rate on plateau

In [7]:
def custom_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.01) 

    
    
required_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience = 30, restore_best_weights = True),
    tf.keras.callbacks.LearningRateScheduler(custom_scheduler),
    tf.keras.callbacks.ReduceLROnPlateau(factor = 0.5, patience = 10, verbose = 1, cooldown = 5, min_lr = 0.0000001),
    tf.keras.callbacks.TerminateOnNaN(),
    tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto'),
    tf.keras.callbacks.CSVLogger('./MT-Resnet/model_history_log_1.csv', append=True)

]

# 4. Train Model

In [8]:
continue_train = False
if continue_train:
    model = load_model(model_path)
else:
    model = MT_ResNet()

In [9]:
Loss_Functions = {'output_1': 'mse', 'output_2': 'binary_crossentropy'}
Loss_Weights = {'output_1': 2, 'output_2': 1}
Metrics= {'output_1': 'mse', 'output_2': 'accuracy'}
adam_optimizer = Adam(learning_rate = 0.1)
model.compile(optimizer = adam_optimizer, loss = Loss_Functions, loss_weights = Loss_Weights, metrics = Metrics)
history = model.fit(img_full_train, [y_full_train[:, 0], y_full_train[:, 2]], batch_size = 20, epochs = 1000, verbose = 1, 
          callbacks = required_callbacks, validation_split = 0.2)

Epoch 1/1000
132/132 [==============================] - ETA: 0s - loss: 1035.0428 - output_1_loss: 516.9102 - output_2_loss: 1.2221 - output_1_mse: 516.9102 - output_2_accuracy: 0.5280
Epoch 1: val_loss improved from inf to 1946561152.00000, saving model to ./MT-Resnet/


INFO:tensorflow:Assets written to: ./MT-Resnet/assets


INFO:tensorflow:Assets written to: ./MT-Resnet/assets


132/132 [==============================] - 142s 971ms/step - loss: 1035.0428 - output_1_loss: 516.9102 - output_2_loss: 1.2221 - output_1_mse: 516.9102 - output_2_accuracy: 0.5280 - val_loss: 1946561152.0000 - val_output_1_loss: 973274432.0000 - val_output_2_loss: 12165.3633 - val_output_1_mse: 973274432.0000 - val_output_2_accuracy: 0.5182 - lr: 0.1000
Epoch 2/1000
132/132 [==============================] - ETA: 0s - loss: 2.1022 - output_1_loss: 0.6407 - output_2_loss: 0.8208 - output_1_mse: 0.6407 - output_2_accuracy: 0.5697
Epoch 2: val_loss improved from 1946561152.00000 to 371177.34375, saving model to ./MT-Resnet/


INFO:tensorflow:Assets written to: ./MT-Resnet/assets


...

INFO:tensorflow:Assets written to: ./MT-Resnet/assets


INFO:tensorflow:Assets written to: ./MT-Resnet/assets


132/132 [==============================] - 128s 970ms/step - loss: 0.3059 - output_1_loss: 0.1192 - output_2_loss: 0.0675 - output_1_mse: 0.1192 - output_2_accuracy: 0.9780 - val_loss: 0.8147 - val_output_1_loss: 0.2542 - val_output_2_loss: 0.3064 - val_output_1_mse: 0.2542 - val_output_2_accuracy: 0.9045 - lr: 0.0352
Epoch 46/1000
132/132 [==============================] - ETA: 0s - loss: 0.2657 - output_1_loss: 0.1006 - output_2_loss: 0.0644 - output_1_mse: 0.1006 - output_2_accuracy: 0.9773
Epoch 46: val_loss did not improve from 0.81469
132/132 [==============================] - 103s 778ms/step - loss: 0.2657 - output_1_loss: 0.1006 - output_2_loss: 0.0644 - output_1_mse: 0.1006 - output_2_accuracy: 0.9773 - val_loss: 1.0641 - val_output_1_loss: 0.3091 - val_output_2_loss: 0.4458 - val_output_1_mse: 0.3091 - val_output_2_accuracy: 0.8773 - lr: 0.0349
Epoch 47/1000
132/132 [==============================] - ETA: 0s - loss: 0.2582 - output_1_loss: 0.0985 - output_2_loss: 0.0612 - out

132/132 [==============================] - ETA: 0s - loss: 0.1160 - output_1_loss: 0.0543 - output_2_loss: 0.0074 - output_1_mse: 0.0543 - output_2_accuracy: 0.9989
Epoch 60: val_loss did not improve from 0.81469
132/132 [==============================] - 104s 792ms/step - loss: 0.1160 - output_1_loss: 0.0543 - output_2_loss: 0.0074 - output_1_mse: 0.0543 - output_2_accuracy: 0.9989 - val_loss: 0.9405 - val_output_1_loss: 0.2614 - val_output_2_loss: 0.4177 - val_output_1_mse: 0.2614 - val_output_2_accuracy: 0.8788 - lr: 0.0152
Epoch 61/1000
132/132 [==============================] - ETA: 0s - loss: 0.1141 - output_1_loss: 0.0534 - output_2_loss: 0.0072 - output_1_mse: 0.0534 - output_2_accuracy: 0.9977
Epoch 61: val_loss did not improve from 0.81469
132/132 [==============================] - 105s 793ms/step - loss: 0.1141 - output_1_loss: 0.0534 - output_2_loss: 0.0072 - output_1_mse: 0.0534 - output_2_accuracy: 0.9977 - val_loss: 0.9293 - val_output_1_loss: 0.2646 - val_output_2_loss:

Epoch 75/1000
132/132 [==============================] - ETA: 0s - loss: 0.0633 - output_1_loss: 0.0307 - output_2_loss: 0.0019 - output_1_mse: 0.0307 - output_2_accuracy: 1.0000
Epoch 75: val_loss did not improve from 0.81469
132/132 [==============================] - 102s 770ms/step - loss: 0.0633 - output_1_loss: 0.0307 - output_2_loss: 0.0019 - output_1_mse: 0.0307 - output_2_accuracy: 1.0000 - val_loss: 0.8836 - val_output_1_loss: 0.2587 - val_output_2_loss: 0.3662 - val_output_1_mse: 0.2587 - val_output_2_accuracy: 0.9121 - lr: 0.0065


In [36]:
model.save('./MT-Resnet/final')

INFO:tensorflow:Assets written to: ./MT-Resnet/final/assets


INFO:tensorflow:Assets written to: ./MT-Resnet/final/assets


# 5. Model Evaluation

## 5.1 Performance on SCUT-FBP Test Dataset

In [37]:
results = model.evaluate(img_test, [y_test[:,0], y_test[:,2]], batch_size=20)
print("loss of FAP task: ", results[1])
print("loss of gender prediction task: ", results[2])

110/110 [==============================] - 12s 112ms/step - loss: 1.1057 - output_1_loss: 0.3801 - output_2_loss: 0.3455 - output_1_mse: 0.3801 - output_2_accuracy: 0.8882
loss of FAP task:  0.38012954592704773
loss of gender prediction task:  0.34548962116241455
accuracy of gender prediction task:  0.8881818056106567


In [41]:
def prob2label(y_pred):
    y_pred_2 = []
    for arr in y_pred[1]:
        if arr[0] >= 0.5:
            y_pred_2.append(1)
        else:
            y_pred_2.append(0)
    
    return y_pred_2

In [44]:
y_pred = model.predict(img_test, batch_size=20)
y_pred_2 = prob2label(y_pred)
print("accuracy of gender prediction task: ", accuracy_score(y_true=y_test[:,2], y_pred=y_pred_2))

110/110 [==============================] - 13s 113ms/step
accuracy of gender prediction task:  0.8881818181818182


## 5.2 Performance on MEBeauty Test Dataset

In [38]:
# load data: y_arr = [rating, race, gender]
def load_data_MEB(label_dir):
    all_ratings = pd.read_csv(label_dir, sep = ' ', header = None)
    all_ratings.columns = ['img_path', 'rating']

    img_arr = np.zeros([len(all_ratings), 64, 64, 3])
    y_arr = np.zeros([len(all_ratings), 3])

    for i in tqdm(range(len(all_ratings))):
    #for i in tqdm(range(500)):
        file_name = all_ratings.iloc[i, 0]
        if file_name.split('/')[3] == 'male':
            y_arr[i, 2] = 0
        else:
            y_arr[i, 2] = 1   
        y_arr[i, 0] = all_ratings.iloc[i, 1] / 2
    
        img = tf.io.read_file(file_name)
        img = tf.image.decode_jpeg(img, channels = 3)
        img = tf.keras.layers.Resizing(64, 64)(img)
        img = preprocess_input(img)
        img_arr[i] = img

    return img_arr, y_arr

img_test_meb, y_test_meb = load_data_MEB('./test_2022.txt')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 528/528 [00:05<00:00, 100.64it/s]


In [40]:
results_meb = model.evaluate(img_test_meb, [y_test_meb[:,0], y_test_meb[:,2]], batch_size=20)
print("loss of FAP task: ", results_meb[1])
print("loss of gender prediction task: ", results_meb[2])
print("accuracy of gender prediction task: ", results_meb[4])

27/27 [==============================] - 3s 111ms/step - loss: 2.8950 - output_1_loss: 0.8137 - output_2_loss: 1.2677 - output_1_mse: 0.8137 - output_2_accuracy: 0.6742
loss of FAP task:  0.8136701583862305
loss of gender prediction task:  1.2676615715026855
accuracy of gender prediction task:  0.6742424368858337


In [46]:
y_pred = model.predict(img_test_meb, batch_size=20)
y_pred_2 = prob2label(y_pred)
print("accuracy of gender prediction task: ", accuracy_score(y_true=y_test_meb[:,2], y_pred=y_pred_2))

27/27 [==============================] - 3s 110ms/step
accuracy of gender prediction task:  0.6742424242424242
